In [2]:
!pip install datasets
!pip install transformers
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

dataset['train'][0]

{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}

In [22]:
from transformers import AutoTokenizer
model_checkpoint = "google-bert/bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_data = dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [42]:
train_dataset = tokenized_data['train'].shuffle().select(range(1000)) #smaller amt of data since it takes a lot of processing power
test_dataset = tokenized_data['test'].shuffle().select(range(1000))

In [54]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels = 5)
TrainingArgs = TrainingArguments(
    output_dir='/content/drive/results',          # output directory
    do_predict= True,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=2,   # batch size for evaluation
    warmup_steps=15,                # number of warmup steps for learning rate
    save_steps=30,
    save_total_limit=10,
    load_best_model_at_end= True,
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=30,
    eval_strategy = 'steps')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
! pip install -U accelerate
! pip install -U transformers

In [55]:
import numpy as np
import evaluate

metric = evaluate.load('accuracy')
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  pred = np.argmax(logits, axis = -1)
  return metric.compute(predictions = pred, references = labels)
trainer = Trainer(
    model = model,
    args = TrainingArgs,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics)

In [56]:
trainer.train()


Step,Training Loss,Validation Loss,Accuracy
30,1.696600,1.648180,0.205000
60,1.611100,1.549993,0.312000
90,1.563400,1.425337,0.391000
120,1.438900,1.286613,0.429000
150,1.350700,1.229219,0.466000
180,1.307200,1.225120,0.435000
210,1.234600,1.349812,0.412000
240,1.388400,1.169273,0.515000
270,1.074800,1.225482,0.474000
300,1.020000,1.269117,0.475000


TrainOutput(global_step=750, training_loss=1.0391238606770834, metrics={'train_runtime': 1355.7746, 'train_samples_per_second': 2.213, 'train_steps_per_second': 0.553, 'total_flos': 789354427392000.0, 'train_loss': 1.0391238606770834, 'epoch': 3.0})

In [57]:
print(test_dataset)
trainer.evaluate(test_dataset)

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})


{'eval_loss': 1.0462288856506348,
 'eval_accuracy': 0.565,
 'eval_runtime': 30.6513,
 'eval_samples_per_second': 32.625,
 'eval_steps_per_second': 16.313,
 'epoch': 3.0}

In [58]:
trainer.save_model("./my_model")

In [60]:
from huggingface_hub import notebook_login
notebook_login()

In [61]:
trainer.push_to_hub('Kirkos27/ModelFor_yelp_review_full')

training_args.bin:   0%|          | 0.00/5.05k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Kirkos27/results/commit/78ad6fd240068b0caab38fd0c35c6a23991a87e7', commit_message='Kirkos27/ModelFor_yelp_review_full', commit_description='', oid='78ad6fd240068b0caab38fd0c35c6a23991a87e7', pr_url=None, pr_revision=None, pr_num=None)